In [1]:
import cv2
import mediapipe as mp
import numpy as np
import time 
import pyrealsense2 as rs
import math as m
import matplotlib.pyplot as plt
from PIL import Image
import time
from scipy import stats

In [2]:
pos_dict = {"nose":0,
"left eye (inner)":1,
"left eye":2,
"left eye (outer)":3,
"right eye (inner)":4,
"right eye":5,
"right eye (outer)":6,
"left ear":7,
"right ear":8,
"mouth (left)":9,
"mouth (right)":10,
"left shoulder":11,
"right shoulder":12,
"left elbow":13,
"right elbow":14,
"left wrist":15,
"right wrist":16,
"left pinky":17,
"right pinky":18,
"left index":19,
"right index":20,
"left thumb":21,
"right thumb":22,
"left hip":23,
"right hip":24,
"left knee":25,
"right knee":26,
"left ankle":27,
"right ankle":28,
"left heel":29,
"right heel":30,
"left foot index":31,
"right foot index":32}

## Using mediapipe pose

In [3]:
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles
mp_pose = mp.solutions.pose

new_frame_time = 0
prev_frame_time = 0

# For webcam input:
cap = cv2.VideoCapture(0)
with mp_pose.Pose(
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5) as pose:
  while cap.isOpened():
    success, image = cap.read()
    if not success:
      print("Ignoring empty camera frame.")
      # If loading a video, use 'break' instead of 'continue'.
      continue

    new_frame_time = time.time()
    fps = 1/(new_frame_time-prev_frame_time)
    prev_frame_time = new_frame_time

    # To improve performance, optionally mark the image as not writeable to
    # pass by reference.
    image.flags.writeable = False
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    results = pose.process(image)

    # # compute the commands based on the angles of the arms at the shoulder
    # left_angle, right_angle = compute_arms_angle(results.pose_landmarks.landmark)
    # command = angles_to_command(left_angle, right_angle)
    # print(command)

    # Draw the pose annotation on the image.
    image.flags.writeable = True
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
    mp_drawing.draw_landmarks(
        image,
        results.pose_landmarks,
        mp_pose.POSE_CONNECTIONS,
        landmark_drawing_spec=mp_drawing_styles.get_default_pose_landmarks_style())
    # Flip the image horizontally for a selfie-view display.
    fps = str(int(fps))
    image = cv2.flip(image,1)
    cv2.putText(image, fps, (7, 70), 1, 3, (100, 255, 0), 3, cv2.LINE_AA)
    cv2.imshow('MediaPipe Pose', image)
    c = cv2.waitKey(1)
    if c == 27: # press escape to quit
        break
cap.release()
cv2.destroyAllWindows()

INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


## Using mediapipe Hollistic which finds keypoints for the pose, face landmarks and hand keypoints

Runs at about 15 fps inference is done on CPU

In [6]:
def convert_landmark_to_array(landmarks):
    landmarks_array = np.array(
        [[landmark.x, landmark.y, landmark.z] for landmark in landmarks])
    return landmarks_array

In [7]:
def normalize_data(data):
    for i in range(3):
        channel = data[:,:,i]
        mean = np.mean(channel)
        std = np.std(channel)
        if std != 0: #to avoid dividing by 0 and having nan values in the arrays
            data[:,:,i] = (channel - mean) / std


In [10]:
cap.release()
cv2.destroyAllWindows()

In [13]:
new_frame_time = 0
prev_frame_time = 0

RUN_NUMBER = 5 # we will save the data in a folder with this number

# could find a way of finding this value automatically
frames_by_sec = 20 - 1
n_frames = -1
landmark_image = np.empty((75,3))

#hollistic detects posiiton, face and hand landmarks
mp_holistic = mp.solutions.holistic
holistic_model = mp_holistic.Holistic(
    model_complexity=1,
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5
)

mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles

moves = ["sit", "lie down", "stay", "come"]

n_samples = 30 # number of times we will record each move
current_sample = 0 # current sample we are recording
current_move = 0

cap = cv2.VideoCapture(0)
cv2.namedWindow("image", cv2.WINDOW_NORMAL)
while cap.isOpened():
    success, image = cap.read()
    if not success:
      print("Ignoring empty camera frame.")
      # If loading a video, use 'break' instead of 'continue'.
      continue

    if n_frames == frames_by_sec or n_frames == -1: # we make 1 sec recordings -> save the data , reset the counter, pause for 2 seconds and update the sample and move counter

        # save and reset the "image" array
        if n_frames != -1:
          landmark_image = np.transpose(landmark_image, (0, 2, 1))
          # normalize_data(landmark_image)
          np.save(f"data_test/{moves[current_move]}/{RUN_NUMBER}{current_sample}", landmark_image)
          landmark_image = np.empty((75,3))

        n_frames = 0
        current_sample += 1
        
        # display which action will be recorded next
        image = cv2.flip(image,1)
        cv2.putText(image, f"prepare for {moves[current_move]} sample no {current_sample}", (7, 70), fontScale = 2, fontFace = 1, color = (100, 255, 0), thickness = 2, lineType = cv2.LINE_AA)
        if current_sample == n_samples:
            current_sample = 0
            current_move += 1
            if current_move == len(moves):
               break

        cv2.imshow("image", image)
        cv2.waitKey(2000)
        continue

    n_frames += 1

    # compute fps
    new_frame_time = time.time()
    fps = 1/(new_frame_time-prev_frame_time)
    prev_frame_time = new_frame_time

    # compute keypoints etc...
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
    image.flags.writeable = False
    results = holistic_model.process(image)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    # convert landmarks to array with exception handling (zero array if not detected)
    try:
      pose_landmarks = convert_landmark_to_array(results.pose_landmarks.landmark)
    except:
      #  print("no pose landmarks detected")
       pose_landmarks = np.zeros((33,3))
    try:
      left_hand_landmarks = convert_landmark_to_array(results.left_hand_landmarks.landmark)
    except:
      # print("no left hand landmarks detected")
      left_hand_landmarks = np.zeros((21,3))
    try:
      right_hand_landmarks = convert_landmark_to_array(results.right_hand_landmarks.landmark)
    except:
      # print("no right hand landmarks detected")
      right_hand_landmarks = np.zeros((21,3))

    hands_and_pose_array = np.concatenate((pose_landmarks, left_hand_landmarks, right_hand_landmarks), axis=0)
    landmark_image = np.dstack((landmark_image, hands_and_pose_array))

    # Drawing the annotiations
    image.flags.writeable = True
    
    # Drawing Pose Landmarks
    mp_drawing.draw_landmarks(
        image,
        results.pose_landmarks,
        mp_holistic.POSE_CONNECTIONS,
        landmark_drawing_spec=mp_drawing_styles.get_default_pose_landmarks_style())
    
    # Drawing Right hand Landmarks
    mp_drawing.draw_landmarks(
      image,
      results.right_hand_landmarks,
      mp_holistic.HAND_CONNECTIONS
    )
 
    # Drawing Left hand Landmarks
    mp_drawing.draw_landmarks(
      image,
      results.left_hand_landmarks,
      mp_holistic.HAND_CONNECTIONS
    )

    fps = str(int(fps))
    image = cv2.flip(image,1)
    cv2.putText(image, fps, (7, 70), 1, 3, (100, 255, 0), 3, cv2.LINE_AA)
    cv2.imshow('image', image)
    c = cv2.waitKey(1)
    if c == 27: # press escape to quit
        break
cap.release()
cv2.destroyAllWindows()